<a href="https://colab.research.google.com/github/SyedWaqar-ah/practiceInColab/blob/master/TITANIC_(Kaggle_challenge).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Challenge 1:
## TITANIC; passenger survival prediction

First let us load the data given already in csv files

In [0]:
#importing some necessary stuff for processing

import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

## Uploading files in COLAB
#### Import from your local drive


In [2]:
#importing train file
import io
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
# reading the file in dataframe

train = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))

In [4]:
#importing train file

uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
test = pd.read_csv(io.StringIO(uploaded['test.csv'].decode('utf-8')))

## Uploading local files in locally ran jupyter notebook
Make sure the files are in same path as your notebook, otherwise you have to provide precise location of file on you local drive

In [0]:
# In case you are locally running this jupyter notebook, unhash the following code

#train = pd.read_csv("train.csv")
#test = pd.read_csv("test.csv")
#test_sta = pd.read_csv("gender_submission")


In [6]:
# viewing first few rows of the training file
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
len(test), len(train)

(418, 891)

so we see that the data given is already divided in testing and training files with 418 and 891 passenger's data, respectively. Now, we know that a phenomenon such as surviving an accedient is not necessarily random and we also know that the crew used the window of opportunity to organize emergency protocol and used available resources to save a preferrable group. This induces the biasness and allows for the application of 'learning' this pattern and predicting a survival probability for a given passenger. 

let us view first hand correlation among the given factors

In [9]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


we will only focus on the row 'Survived'. We see that this feature has some useless relations such as correlation with passengerId, SibSp (number of sibling and spouse), Parch (parents) and the fare. Also note that the columns Name, Sex, Ticket, Cabin and Embarked are missing from this table, this is due to the fact that their values are in string format, so thats fine but remember the biasness we ought to be careful with includes the feature of gender. So we must include this column and one way to that is by associating values of 0 and 1 (or 1 and 0) for male and female, respectively.

In [10]:
train.Sex = train.Sex.replace(['male','female'],[1,0])
train.Sex

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [11]:
# this is how the data looks like
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",1,26.0,0,0,111369,30.0000,C148,C


In [12]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.369226,0.083081,0.018443,-0.549500
Sex,0.042939,-0.543351,0.131900,1.000000,0.093254,-0.114631,-0.245489,-0.182333
Age,0.036847,-0.077221,-0.369226,0.093254,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.114631,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.245489,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,-0.182333,0.096067,0.159651,0.216225,1.000000


Observe the negative sign in correlation value -0.543351 between Survived and Sex, it is only due to our choice of substituting the value of 1 with male and 0 with female. This also highlights that the number of female survivors is greater than number of male survivors in the incident.
let us check if our inference is correct...

In [13]:
#passengers on board
train.Sex.value_counts()
#1-male
#0-female

1    577
0    314
Name: Sex, dtype: int64

In [14]:
#number of total survivors
train.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

so 342 passengers survived out of 891, (remember this is not complete data) the ratio being 38%. Male to female ratio was 1.838 for onboard passengers. 

In [0]:
#number of survived males and females









we should go ahead and do the same for test data set.

In [0]:
test.Sex = test.Sex.replace(['male','female'],[1,0])

In [16]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",1,NaN,0,0,359309,8.0500,NaN,S


things for data analysis:
number of male and female passegers
number of male and female survivers
average age
age groups
average servival age
number of passengers in each class


## Preparing the Training data

The method I'd like to use is to first get rid of all the useless feature and form one dataframe with features contributing towards survival probability with our set presumptions.  This can be done by dropping unconcerned columns,

In [0]:
#data preparation: filteration
train_fea = train.drop(['Name','PassengerId','SibSp','Parch','Ticket','Fare','Cabin','Embarked','Survived'] , axis=1)

train_stat = [train.Survived]

In [0]:
#data preparation: values adjustment
# to replace any unknown values (NaN) with zero (zero will be processable, unlike NaN)
train_fea = train_fea.fillna(0)

In [19]:
train_fea

,Pclass,Sex,Age
0,3,1,22.0
1,1,0,38.0
2,3,0,26.0
3,1,0,35.0
4,3,1,35.0
...,...,...,...
886,2,1,27.0
887,1,0,19.0
888,3,0,0.0
889,1,1,26.0


Note that we have left out the survival column as well, this is due to the fact that survival colums is a status (label) for a given passenger, not a feature.
So far, our datasets are in the form of dataframes (pandas objects containing sequence of indexed lists). In order to feed these values in a neural layer we need to form array (n-dimensional numpy objects; tensors). We will call this array 'train_feature'

In [0]:
train_feature = np.array([train_fea.Sex, train_fea.Pclass, train_fea.Age])

In [21]:
train_feature

array([[ 1.,  0.,  0., ...,  0.,  1.,  1.],
       [ 3.,  1.,  3., ...,  3.,  1.,  3.],
       [22., 38., 26., ...,  0., 26., 32.]])

shape of this array is...

In [22]:
train_feature.shape

(3, 891)

Neural layers accept data for each object in the dataset, we have in our training dataset 891 objects(passengers), and the input values are features which we have reduced to 3. With shape (3,891) neural layer will be fed first vector first that means it will encounter 891 values of same feature to process, which is dead wrong. We can correct this by transposing the dataset all together.

In [23]:
train_feature = train_feature.T
train_feature

array([[ 1.,  3., 22.],
       [ 0.,  1., 38.],
       [ 0.,  3., 26.],
       ...,
       [ 0.,  3.,  0.],
       [ 1.,  1., 26.],
       [ 1.,  3., 32.]])

In [24]:
train_feature.shape

(891, 3)

now it looks to be in good shape!


We can now do the same process for labels (passenger status)...

In [25]:
train_status = np.array(train_stat)
train_status

array([[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 

In [26]:
train_status.shape

(1, 891)

again, readjusting the shape...

In [0]:
train_status = train_status.T

In [28]:
train_status.shape

(891, 1)

The processes we went above is called "feature engneering". And yes, it depends on what type of data we are given. Just to admire the next step, lets take a look at what neural net will see when we feed in the train_feature array one vector at a time.

In [29]:
train_feature[1]
#i.e. the passenger is 'female', 'first' class passenger of age '38'

array([ 0.,  1., 38.])

In [70]:
#and did she survived???
train_status[1]

array([1])

well, yes she did.

## Preparaing testing data
Similar to training data the testing data needs to be filtered, put in array form for both features and status. So the following codes will prepare testing data with the same logic as for training data.

In [0]:
#data preparation: filteration
test_fea = test.drop(['Name','PassengerId','SibSp','Parch','Ticket','Fare','Cabin','Embarked'] , axis=1)
#test_sta = test_sta.drop(['PassengerId'], axis=1)

In [0]:
#data preparation: values adjustment
# to replace any unknown values (NaN) with zero (zero will be processable, unlike NaN)
test_fea = test_fea.fillna(0)

In [0]:
#data preparation: forming array
test_feature = np.array([test_fea.Sex, test_fea.Pclass, test_fea.Age])
#test_status = np.array([test_sta.Survived])
#transposing the array
test_feature = test_feature.T
#test_status = test_survive.T

In [35]:
test_feature

array([[ 1. ,  3. , 34.5],
       [ 0. ,  3. , 47. ],
       [ 1. ,  2. , 62. ],
       ...,
       [ 1. ,  3. , 38.5],
       [ 1. ,  3. ,  0. ],
       [ 1. ,  3. ,  0. ]])

In [0]:
#test_status

## continous Validation method

In [0]:
import tensorflow as tf
from keras import models
from keras import layers


In [0]:
num_fea = 3


def tt_model():
    model = models.Sequential()
    model.add(layers.Dense(num_fea, activation='relu', input_shape=(train_feature.shape[1])))
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dense(1, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy'])
    return model

In [110]:
model.fit(train_feature, train_status, epochs=10, batch_size=128, validation_data=(test_feature, test_status))

AttributeError: ignored

## k-fold cross-validation method

In [0]:
from keras import models
from keras import layers

def ttk_model():
    # k stands for k-folds
    model = models.Sequential()
    model.add(layers.Dense(3, activation='relu',input_shape=(train_feature.shape[1])))
    #model.add(layers.Dense(2, activation='relu'))
    model.add(layers.Dense(1, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [91]:
k = 3
num_val_samples = len(train_feature) // k
num_epochs = 4
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_feature = train_feature[i * num_val_samples: (i + 1) * num_val_samples]
    val_status = train_status[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_feature = np.concatenate([train_feature[:i * num_val_samples],train_feature[(i + 1) * num_val_samples:]],axis=0)
    partial_train_status = np.concatenate([train_status[:i * num_val_samples],train_status[(i + 1) * num_val_samples:]],axis=0)
    
    model = ttk_model()
    model.fit(partial_train_feature, partial_train_status, epochs=num_epochs, batch_size=1, verbose=0)
    
    val_mse, val_mae = model.evaluate(val_feature, val_status, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2


In [92]:
all_scores

[0.6430976439003993, 0.5690235696256362, 0.6363636367650144]

In [87]:
np.mean(all_scores)

0.6161616167636832

In [0]:
test_p_status = ttk_model().predict(test_feature)

In [95]:
test_p_status

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [61]:
len(test_p_status)

418